In [ ]:
import pandas as pd
import numpy as np
import cv2
from keras.models import load_model
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import normalize
from skimage import measure, color, io
from PIL import Image

In [ ]:
model = load_model('400-full_best_model.h5')
img=load_img('train/image/image/144.jpg',target_size=(256,256)) 
img=img_to_array(img)
img = img/255
test_img_norm=img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
plt.imsave('output.jpg', prediction, cmap='gray')

In [ ]:
prediction = cv2.imread('output.jpg',0)
ret1, thresh = cv2.threshold(prediction, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
sure_bg = cv2.dilate(opening,kernel,iterations=10)
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret2, sure_fg = cv2.threshold(dist_transform, 0.26*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
ret3, markers = cv2.connectedComponents(sure_fg)
markers = markers+10
markers[unknown==255] = 0


props = measure.regionprops_table(markers, intensity_image=prediction, 
                              properties=['label',
                                          'area', 'equivalent_diameter',
                                          'mean_intensity', 'solidity', 'orientation'])

df = pd.DataFrame(props)
df